In [ ]:
import numpy as np
import numpy.ma as ma
import cv2
from matplotlib import pyplot as plt

from skimage import filters

In [ ]:
raw_data_dir = "raw_data/"

In [ ]:
def show_color_im(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
img = cv2.imread(raw_data_dir + '2p22_00.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

thresh_val = np.median(gray)/1.15
ret, thresh = cv2.threshold(gray,thresh_val,255,cv2.THRESH_BINARY_INV)
plt.imshow(thresh)
plt.show()

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
plt.imshow(opening)
plt.show()

In [ ]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)
plt.imshow(sure_fg)
plt.show()

In [ ]:
mask = cv2.cvtColor(sure_fg,cv2.COLOR_GRAY2RGB)
img_masked = img.copy()
img_masked[mask==0] = 0
show_color_im(img_masked)
plt.show()
show_color_im(img)
plt.show()

In [ ]:
sobel = filters.sobel(gray)
blurred = filters.gaussian(sobel, sigma=2.0)
plt.imshow(blurred)
plt.show()

In [ ]:
import numpy as np
light_spots = np.array((gray > 230).nonzero()).T
plt.plot(light_spots[:, 1], light_spots[:, 0], 'o')
plt.imshow(gray)
plt.show()

In [ ]:
dark_spots = np.array((gray < 95).nonzero()).T
plt.plot(dark_spots[:, 1], dark_spots[:, 0], 'o')
plt.imshow(gray)
plt.show()

In [ ]:
from scipy import ndimage as ndi
bool_mask = np.zeros(gray.shape, dtype=np.bool)
bool_mask[tuple(light_spots.T)] = True
bool_mask[tuple(dark_spots.T)] = True
seed_mask, num_seeds = ndi.label(bool_mask)
num_seeds

In [ ]:
from skimage import morphology
ws = morphology.watershed(blurred, seed_mask)
plt.imshow(ws)
plt.show()

In [ ]:
background = max(set(ws.ravel()), key=lambda g: np.sum(ws == g))
background_mask = (ws == background)
plt.imshow(~background_mask)
plt.show()